INFO-H-515 Project <br>
2022–2023

# Phase 2 : Preprocessing
Gianluca Bontempi, Théo Verhelst, Cédric Simar <br>
Computer Science Department, ULB

## Approach

The idea is to create the perfect table in python and then turn it into a spark dataframe. As such, for each sensor, we can merge the perfect table with 

## Imports and configuration

In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col, rank
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType
import pandas as pd
import os
import datetime

In [22]:
spark = SparkSession \
    .builder \
    .master("local[10]")\
    .config("spark.executor.instances", "1") \
    .config("spark.executor.cores", "10") \
    .config("spark.executor.memory", "16G") \
    .appName("¨Preprocessing") \
    .getOrCreate()

# Let us retrieve the sparkContext object
sc=spark.sparkContext

## Obtaining the data

The following function is running the script `getdata.py` which is querying the data concurrently. It first get the sensor from the API, then it loads the sensors, put them into an RDD and parallelizing the query of the results. NB : there is a known bug for users on Mac OS with M-chips. The latter doesn't allow slaves from spark to make external requests. In the docker image, the function run fine, except two warnings in the end.

In [ ]:
!python getdata.py

## Adding the IDs in the csv

The first step is to add to each file the sensor_id.

In [23]:
PATH = "data/sensors"
OUT_dir = "data/processed"

if not os.path.exists(OUT_dir):
    os.makedirs(OUT_dir)

files = [os.path.join(PATH, f) for f in os.listdir(PATH)]

def add_sensor_information(filename, output_dir) -> None:
    sensor_name = filename.split('.')[0].split("_")[-1]
    sensor_data = pd.read_csv(filename)
    sensor_data["sensor"] = sensor_name
    sensor_data.to_csv(f'{output_dir}/{sensor_name}.csv', index=False)

parallel_files = sc.parallelize(files)

parallel_files.foreach(lambda x : add_sensor_information(x, OUT_dir))


## Procesing the missing timestamps

The discrepencies in the format for missing data can be solved by creating a perfect table, on which the real data will be joined. Missing records will be then just filled.

In [24]:
files = [os.path.join(OUT_dir, f) for f in os.listdir(OUT_dir)]
sensors = [f.split(".")[0].split("/")[-1] for f in files]

#### Generating the perfect table

In [25]:
def generate_days(start_date: datetime.date, end_date: datetime.date) -> list[str]:
    """
    Generates a list of dates between two dates.

    Args:
        start_date (datetime.date): The start date of the range.
        end_date (datetime.date): The end date of the range.

    Returns:
        list[str]: A list of dates between the start and end dates.
    """
    days = []
    current_date = start_date
    while current_date <= end_date:
        days.append(current_date.strftime("%Y-%m-%d"))
        current_date += datetime.timedelta(days=1)
    return days

intervals = 96

# Generate a list of dates between the specified start and end dates
days = generate_days(datetime.date(2018, 12, 6), datetime.date(2023, 3, 31))

# Create a perfect frame by combining each date with a range of intervals
perfect_frame = [[day, i] for day in days for i in range(1, intervals+1)]

def add_sensor(x: list, sensors: list) -> list:
    """
    Adds sensor information to the input list.

    Args:
        x (list): The input list to which sensor information will be added.
        sensors (list): The list of sensors to add.

    Returns:
        list: The input list with sensor information added.
    """
    return [x + [sensor] for sensor in sensors]

# Create an RDD from the perfect_frame with only the dates to add the sensors
rdd_pf = sc.parallelize(perfect_frame)\
    .flatMap(lambda x: add_sensor(x, sensors))\
    .map(lambda x: (x[0]+ "-" + str(x[1]) + "-" + str(x[2]), x))

#### Adding missing timestamps

In [26]:
raw_files = sc.textFile(OUT_dir)\
    .map(lambda x: x.split(','))\
    .filter(lambda x: x[0] != "Date")\
    .map(lambda x: (x[0]+ "-" + str(x[1]) + "-" + str(x[-1]), x))

def validate_results(x):
    """
    Validates the results by checking if the join could have been made.
    In the other case, there was an empty record, and we can fill it.

    Args:
        x: The input tuple containing the key-value pair.

    Returns:
        list: The validated results.
    """
    if x[1] == None:
        return [x[0][0], int(x[0][1]), 0, -1] + x[0][2:]
    else :
        x[1][1] = int(x[1][1])
        x[1][2] = int(x[1][2])
        x[1][3] = int(x[1][3])
        return x[1]
    
processed_files = rdd_pf.leftOuterJoin(raw_files)\
                    .mapValues(validate_results)\
                    .map(lambda x: x[1])

#### Adding time index

This extra step is to add a unique timestamp that is used as counter to ease the computation later on. This is step is not necessary, this column could be added while loading the data.

In [27]:
columns = ['Date', 'Time Gap', 'Count', 'Average speed', 'sensor']
processed_files_df = spark.createDataFrame(processed_files,columns)\
                                    .sort("Date", "Time Gap", "sensor")

w = Window().partitionBy("sensor").orderBy("Date", "Time Gap")
processed_files_df = processed_files_df.withColumn('timestamp',rank().over(w))

#### Exporting the data

In [28]:
import os
import shutil

def move_csv_files(directory: str, filename: str) -> None:
    """
    Moves all CSV files from the specified directory to a single file with the given filename.
    Deletes the original directory after moving the files.

    Args:
        directory (str): The directory path where the CSV files are located.
        filename (str): The name of the file to which the CSV files will be moved.
    """
    for file in os.listdir(directory):
        if file.endswith(".csv"):
            # Rename each CSV file to the specified filename
            os.rename(os.path.join(directory, file), os.path.join(directory, filename))

            # Move the renamed file to the current directory
            shutil.move(os.path.join(directory, filename), ".")

            # Remove the original directory after moving the file
            shutil.rmtree(directory)

    return None

folder = "bike_counts"

# Write the processed_files_df DataFrame as a single CSV file in the specified folder
processed_files_df.coalesce(1).write.options(header='True', delimiter=',').csv(f"{folder}")

# Move the CSV file(s) in the folder to a single file named "bike_counts.csv"
move_csv_files(f"{folder}", "bike_counts.csv")

## Sanity Checks

In [29]:
schema = StructType([
    StructField("Date", DateType(), nullable = False),
    StructField("Time Gap", IntegerType(), nullable = False),
    StructField("Count", IntegerType(), nullable = False),
    StructField("Average speed", IntegerType(), nullable = False),
    StructField("sensor", StringType(), nullable = False),
    StructField("timestamp", IntegerType(), nullable = False)
    ]) 

bike_counts = spark.read.format("csv") \
        .option("header", True) \
        .schema(schema) \
        .load("bike_counts.csv")\
        .cache()

In [30]:
bike_counts.show()

+----------+--------+-----+-------------+-------+---------+
|      Date|Time Gap|Count|Average speed| sensor|timestamp|
+----------+--------+-----+-------------+-------+---------+
|2018-12-06|       1|    0|           -1|CB02411|        1|
|2018-12-06|       2|    0|           -1|CB02411|        2|
|2018-12-06|       3|    0|           -1|CB02411|        3|
|2018-12-06|       4|    0|           -1|CB02411|        4|
|2018-12-06|       5|    0|           -1|CB02411|        5|
|2018-12-06|       6|    0|           -1|CB02411|        6|
|2018-12-06|       7|    0|           -1|CB02411|        7|
|2018-12-06|       8|    0|           -1|CB02411|        8|
|2018-12-06|       9|    0|           -1|CB02411|        9|
|2018-12-06|      10|    0|           -1|CB02411|       10|
|2018-12-06|      11|    0|           -1|CB02411|       11|
|2018-12-06|      12|    0|           -1|CB02411|       12|
|2018-12-06|      13|    0|           -1|CB02411|       13|
|2018-12-06|      14|    0|           -1

Exactly 18 records per Date per time stamp

In [31]:
bike_counts.groupBy("Date", "Time Gap").count().where((col("count") != 18) ).show()

+----+--------+-----+
|Date|Time Gap|count|
+----+--------+-----+
+----+--------+-----+



Number of record per day for a sensor

In [32]:
bike_counts.groupBy("sensor", "Date").count().where((col("count") != 96)).show()

+------+----+-----+
|sensor|Date|count|
+------+----+-----+
+------+----+-----+



##  Closing the spark session

In [33]:
spark.stop()